# The Flashlight Kernel

In [ ]:
# default_exp data_processing

!nbdev_build_lib

FRED at its core is a diffusion-based model; It embeds directed graphs and high dimensional data into euclidean space with a vector field in terms of probabilities, rather than distances. The Flashlight Kernel is a basis of FRED as it translates points and vectors and points in euclidean space with a vector field into affinities between directed graph nodes.

---

## The Flashlight Kernel

$$k(x_i,x_j) = \exp\left[\frac{-\left(||x_j - x_i||_2 + \beta \cdot \left(||v_i||_2 - \left\langle v_i, \frac{x_j - x_i}{||x_j - x_i||_2}\right\rangle \right)\right)}{\sigma}\right],$$

where $k(x_i,x_j)$ is the affinity from $x_i$ to $x_j$, $v_i$ is the flow at $x_i$ (vector associated with data point in ambient space or vector at point $x_i$ in embedding space with a vector field), and $||\cdot||_2$ is the regular two-norm. $\beta$ and $\sigma$ are tunable hyperparameters, which controls the emphasis or "strength" of flow relative to distance in the calculation of affinities between two points.

---

The Flashlight Kernel shows up in various parts of all iterations and implementations of FRED including but not limited to converting high dimensional input data (data clouds with vectors associated to each data points such as RNA Velocity data) into directed graphs with affinities, converting points in our embedding space (euclidean space with a vector field) into directed graphs with affinities, calculating near neighbors of data points in the ambient space, and more.

The essence of the Flashlight Kernel is to retain asymmetric relationships between data points with directionality similar to that of directed graph nodes. It assigns affinities between two data points by considering both the distance between them and their associated vectors; Going with the flow is highly encouraged and going against or even perpendicular to the flow is discouraged and assigned affinity close to zero. As a result, diffusion or random walks probabilities from data point to data point is heavily dictated by the flow in the embedding space or vectors associated with data points in embient space.

Note that affinities between data points can be (and almost always are) asymmetric i.e., $k(x_i, x_j) \neq k(x_j, x_i)$ as the flow at $x_i$ may be pointing towards $x_j$, but the opposite may not be true. In fact, in single-cell data, this is rarely true as cells do not transition back and forth between two different states.

---

## The Algorithm

$\textbf{Affinitty_from_flow}$

Input:
- F: $n \times d$ tensor with $n$ rows of points each with flow in $d$  column directions
- D: $n \times n \times d$ tensor where dimension 1 represents directions to point i, dimension 2 represents direction from point j, and dimension 3 represents the displacement from point j to point i in each dimension of ($\mathbb{R}^d$). Ex. directions_array\[i\]\[j\] is the vector from point $x_j$ to point $x_i$ i.e., $(x_j - x_i)$.
- b: flow strength ($\beta$)
- s: bandwidth ($\sigma$)

Algorithm:
1. Calculate length of direction vectors, $L$ ($n \times n$).
2. Calculate normalized directions tensor from $D$, $D_n$ (n \times n \times d).
3. Compute dot product between normed directions and flows (Sum $d$ dimensions of $D_nF$ = $C_F$ (cost from flow)).
4. Subtract $C_F$ from length of flows in $F$ ($F_F$, flow factor).
5. Take absoluate value of $F_F$.
6. Add $L$ and $s \cdot F_F$ ($C_{wF}$, cost with flow)
7. Exponentiate $C_{wF}^T$

In [ ]:
# export

def affinity_from_flow(flows, directions_array, flow_strength = 1, sigma=1):
    """Compute probabilities of transition in the given directions based on the flow. 

    Parameters
    ----------
    flow : torch tensor of shape n_points x n_dims
        _description_
    directions_array : torch tensor of shape n_directions x n_points x n_dims. Assumed to be normalized.
        _description_
    sigma : int, optional
        kernel bandwidth, by default 1
    returns (n_points)
    """
    assert len(flow.shape) == 2 # flow should only have one dimension
    assert len(directions_array.shape) > 1 and len(directions_array.shape) < 4
    n_directions = directions_array.shape[0]
    # Normalize directions
    length_of_directions = torch.linalg.norm(directions_array,dim=-1)
    normed_directions = F.normalize(directions_array,dim=-1)
    # and normalize flow # TODO: Perhaps reconsider
    # Calculate flow lengths, used to scale directions to flow
    # flow_lengths = torch.linalg.norm(flow,dim=-1)
    if len(directions_array) == 1: # convert to 2d array if necessary
        directions_array = directions_array[:,None] 
    # scale directions to have same norm as flow
    # scaled_directions = normed_directions * flow_lengths[:,None].repeat(directions_array.shape[0],1,directions_array.shape[2])
    # compute dot products as matrix multiplication
    dot_products = (normed_directions * flow).sum(-1)
    # take distance between flow projected onto direction and the direction
    distance_from_flow = (torch.linalg.norm(flow,dim=1)).repeat(n_directions,1) - dot_products
    # take absolute value
    distance_from_flow = torch.abs(distance_from_flow)
    # print('shape of dff',distance_from_flow.shape)
    # add to this the length of each direction
    distance_from_flow = flow_strength*distance_from_flow + length_of_directions
    # put the points on rows, directions in columns
    distance_from_flow = distance_from_flow.T
    # take kernel of distances
    kernel =  torch.exp(-distance_from_flow/sigma)
    
    return kernel

In [ ]:
# export

def affinity_matrix_from_pointset_to_pointset(pointset1,pointset2,flows,n_neighbors=None,sigma=0.5, flow_strength=1):
    """Compute affinity matrix between the points of pointset1 and pointset2, using the provided flow.

    Parameters
    ----------
    pointset1 : torch tensor, n1 x d
        The first pointset, to calculate affinities *from*
    pointset2 : torch tensor, n2 x d
        The second pointset, to calculate affinities *to* (from pointset1)
    flow : a function that, when called at a point, gives the flow at that point
    n_neighbors : number of neighbors to include in affinity computations. All neighbors beyond it are given affinity zero
    (currently not implemented)

    Returns:
    Affinity matrix: torch tensor of shape n1 x n2
    """
    
    # Calculate the directions from point i in pointset 1 to point j in pointset 2
    n1 = pointset1.shape[0]
    n2 = pointset2.shape[0]
    P2 = pointset2[:,:,None].repeat(1,1,n1)
    P1 = pointset1.T.repeat(n2,1,1)
    P3 = (P2 - P1)
    P3 = P3.transpose(1,2)
    # dimension 1 represents directions to point i
    # dimension 2 represents direction from point j
    # dimension 3 represents direction in each dimension (R^n)
    # compute affinities from flow and directions
    affinities = affinity_from_flow(flows,P3,sigma=sigma,flow_strength=flow_strength)
    
    return affinities